## Exercise 2 ~ The Adventures of the Crypto-Apprentice: Generalized Vernam Cipher With Key Expansion

In [1]:
import re
# Get variable from parameters file
import utils
params = utils.get_parameters()

In [2]:
c2 = params.c2

UTF-16BE adds `\x00` (NULL) between every ASCII character.

This means that every two bytes in the ciphertext, two bites are the "null" character XORed with the cipherkey.

Because of the repeatability over `a` of the key stream, we should be able to find `a` without too much trouble.

In [3]:
s = "Hello !".encode("utf-16be")
print s[0] == '\x00' and s[2] == '\x00'

True


In [4]:
x = bytearray("00ab00aa".decode("hex"))
y = bytearray("00000000".decode("hex"))
z = bytearray(len(x))
for i in range(len(x)):
    z[i] = x[i] ^^ y[i]
    
print z.decode("utf-16be")

«ª


In [5]:
print c2

cc81ccb96605660f3313334599f199f9ccb8ccee6609660d3350335c99b999ebccadccb56646661a3340333999f599e3ccbfcc92664a6613335533f699eb99eccca2cc0966086756331331f799f698bbccecce0a660f6345335639b999ee9ca4cca3c64666147356335e19fc99f48caecca9e6146646335a334799b799b9cc85ccb8664666053356335699f899f6ccecccb9660f6617335b331999ed99f4cca9cce66600661c3341335e99f099e2ccabccc6660966553313336d99f199a9cceccc2166146756335233ed99b9989ecca5ce0866016340331d31b999cd9ca4ccbec60366037313334439fc99eb8ca9ccece601660f3345335619f799b9ccb8cca366466613335b335699b999ddcca0ccb866036610331f331199f099f1cca1cca9661466073352335599b599acccbbcc8f661566563340336d99b999adcca2cc0266466755335233f099eb98a9ccbfce126646675c335531b999f89ca0cca0c64666046356335a39f799fe8cbfcce2e64666357356334519fc99f7ccecccb8660966463347335b99fc99b8cc88ccb9660766153355331c99d599f3ccbecca26615664f3313335e99eb99e9ccadcc926646661e335a337799fc99beccbfcc466607665d335733b999fa98beccadce0066126740335e31fc99f79ceccca3c60066466347335b39fc99b98ca1cca3c613660873473352

In [6]:
# Let's get all possible values of p
possible_primes = [
    p for p in range(1, len(c2) * 4)
        if is_prime(p) and (p - 1) % 16 == 0
]

In [7]:
rec1 = c2[0:2]
rec2 = c2[4:6]
rec3 = c2[8:10]
rec4 = c2[12:14]

pattern = rec1 + '..' + rec2 + '..' + rec3 + '..' + rec4

pattern_data = re.findall(pattern, c2)
pattern_indexes = [c2.find(p) for p in pattern_data]

indexes_difference = []
for idx in range(len(pattern_indexes) - 1):
    indexes_difference.append(pattern_indexes[idx + 1] - pattern_indexes[idx])
    
most_common = max(set(indexes_difference), key=indexes_difference.count)

a = most_common
# a = 4

print "Probable value of a:", most_common

Probable value of a: 32


In [8]:
binary_c2 = bin(int(c2, 16))[2:]
pattern = '0' + binary_c2[0:7] + '.' * 64
bin_data = re.findall(pattern, binary_c2)
bin_indexes = [binary_c2.find(p) for p in bin_data]

In [9]:
def key_builder(p, a, l):
    bit_list = [mod(mod(((8 * i + j + 1) * a), p), 2) for i in range(8 * l) for j in range(8)]
    bit_string = "".join([str(b) for b in bit_list])
    hex_key = '%x' % int(bit_string, 2)
    while len(hex_key) < 16 * l:
        hex_key += '0'
    # hex_key = hex(int(bit_string, 2))
    return bytearray(hex_key.decode("hex"))

plaintext = ''
# decode_length = len(c2)
decode_length = 16

possible = [p for p in bin_indexes if p in possible_primes]
print possible

#for p in possible_primes:
for p in possible:
    for a in range(1, p):
        key = key_builder(p, a, decode_length)
        ciphertext = bytearray(c2.decode("hex"))
        encoded_plaintext = bytearray(decode_length)
        decoded_plaintext = ' '
        if key[0] != ciphertext[0]:
            continue
        for i in range(len(encoded_plaintext)):
            encoded_plaintext[i] = ciphertext[i] ^^ key[i]
        try:
            decoded_plaintext = encoded_plaintext.decode("utf-16be")
        except:
            a
        if decoded_plaintext[0] == '\x00':
            plaintext = decoded_plaintext
            print p, a, plaintext

[193, 3217, 4241, 4481, 7121, 12433, 17713, 20113, 21617, 22769, 25793, 26177, 26417, 29569, 30449]


KeyboardInterrupt: 

### Order of the group is P => all the values except 0 has order P
Key has odd length => I probably see the repetition * 2 (need to compare at bit-level, no byte-level).